In [56]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import smtplib
import os
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import random
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import time
import math
import difflib
import requests
import os
import cv2  # Import OpenCV
import numpy as np
from PIL import Image
# Specify the path to the geckodriver executable
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from PIL import Image
import numpy as np


amazon_titles= []
amazon_links = []
webuy_titles = []
webuy_links = []
amazon_images = []
webuy_images = []
calculated_prices = []
product_codes = []
buy_prices = []


sellers_black_list = [" Live Life Retail"]
seller_white_list = [" Selling-Prime-UK"]
with open("WorkingProxies.txt", "r", encoding="utf-8") as prxsfile:
    proxy_list = prxsfile.read().split("\n")




def get_random_proxy():
    """Select a random proxy from the list."""
    return random.choice(proxy_list)

def create_driver_with_proxy(proxy):
    """Create a new Chrome WebDriver instance with a proxy."""
    # Split the proxy IP and port
    proxy_ip, proxy_port = proxy.split(':')
    
    # Create Chrome options
    chrome_options = Options()
    
    # Configure proxy settings for both HTTP and HTTPS
    proxy_settings = f"{proxy_ip}:{proxy_port}"
    chrome_options.add_argument(f'--proxy-server={proxy_settings}')
    
    # Specify the path to the Chrome executable if needed (optional)
    # chrome_options.binary_location = "C:\\Path\\To\\Chrome.exe"  # Only needed if Chrome is not in PATH
    
    # Initialize the WebDriver with the options
    amazon_driver = webdriver.Chrome(options=chrome_options)
    return amazon_driver

def create_driver_without_proxy():
    """Create a new Chrome WebDriver instance without a proxy."""

    
    # Initialize the WebDriver with the options
    driver = webdriver.Chrome()
    return driver

# Rotate proxies
high_price = 31
low_price = 25
proxy = get_random_proxy()  # Get a random proxy
#driver = create_driver_with_proxy(proxy)  # Create WebDriver with that proxy
amazon_driver = create_driver_without_proxy()
url = f"https://www.amazon.co.uk/s?k=blu+ray&i=merchant-items&me=A1UFLUL80142N4&s=price-desc-rank&qid=1728216348&rnid=389127011&ref=sr_nr_p_36_0_0&low-price={low_price}&high-price={high_price}"

try:
    amazon_driver.get(url)
except Exception:
    print(f"Using proxy: {proxy}, BAD proxy")
# Define a function to wait for an element to be clickable

def wait_for_element_to_be_clickable(driver, xpath, timeout=10):
    return WebDriverWait(driver, timeout).until(
        EC.element_to_be_clickable((By.XPATH, xpath))
    )
amazon_cookies_PATH = '/html/body/div[2]/span/form/div[2]/span[1]/span/input'
try:
    # Wait until the element is clickable
    WebDriverWait(amazon_driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, amazon_cookies_PATH))
    )
    amazon_driver.find_element(By.XPATH, amazon_cookies_PATH).click()
except Exception as e:
    print(f"An error occurred: {e}")
    time.sleep(0.5)

# Click the "Deliver to" button
deliver_to_button_PATH = '//*[@id="glow-ingress-line2"]'
wait_for_element_to_be_clickable(amazon_driver, deliver_to_button_PATH).click()

# Click the Post Code input field
PostCode_input_PATH = '//*[@id="GLUXZipUpdateInput"]'
wait_for_element_to_be_clickable(amazon_driver, PostCode_input_PATH).click()

# Enter the postcode
amazon_driver.find_element(By.XPATH, PostCode_input_PATH).send_keys("SW1A2aa")

# Click the "Apply" button
Apply_button_PATH = '//*[@id="GLUXZipUpdate"]/span/input'
wait_for_element_to_be_clickable(amazon_driver, Apply_button_PATH).click()

# Click the "Continue" button
continue_button_PATH = '/html/body/div[6]/div/div/div[2]/span/span/input'
wait_for_element_to_be_clickable(amazon_driver, continue_button_PATH).click()

# Click to accept cookies
accept_cookies_PATH = '/html/body/div[1]/span/form/div[2]/span[1]/span/input'
wait_for_element_to_be_clickable(amazon_driver, accept_cookies_PATH).click()

try:
    amazon_driver.get(url)
except Exception:
    print(f"Using proxy: {proxy}, BAD proxy")


# Define the XPATH of the target element (next page button)
next_page_button_PATH = 's-pagination-item s-pagination-next s-pagination-button s-pagination-separator'

# Define the scroll step (in pixels)
scroll_step = 500

# Scroll and search for the element
tries = 20  # Number of tries before stopping
i = 0  # Initialize attempt counter
last_height = amazon_driver.execute_script("return document.body.scrollHeight")  # Get the initial page height

while i < tries:
    try:
        html_source = amazon_driver.page_source
        soup = BeautifulSoup(html_source, "html.parser")
        # Try to find the element using its XPATH
        next_button = soup.find_all('svg')[-1]['xmlns']
        if next_button:
            
            break  # Exit the loop if the element is found
        else:
            # Scroll down incrementally by scroll_step
            amazon_driver.execute_script(f"window.scrollBy(0, {scroll_step});")
            
            # Wait for the page to load new content
            time.sleep(1)  # Adjust based on page loading speed
            
            # Get the new scroll height after scrolling
            new_height = amazon_driver.execute_script("return document.body.scrollHeight")
            
            # Check if the page has reached the end (no more content to scroll)
            
            # Update the last height
            last_height = new_height
            i += 1  # Increment attempt counter
    except Exception as error:
        print(error)


Num_of_products = int(amazon_driver.find_element(By.XPATH, "/html/body/div[1]/div[1]/span/div/h1/div/div[1]").text.split("results")[0].strip().split(" ")[-1].replace(",", ''))
print(Num_of_products)
pages = 20 if Num_of_products / 16 >= 20 else  math.ceil(Num_of_products / 16)
print(pages)
products_link_tags = soup.find_all(class_="a-link-normal s-no-outline")
amazon_products_links = ["https://www.amazon.co.uk/"+products_link['href'] for products_link in products_link_tags]

for amazon_products_link in amazon_products_links:
    amazon_links.append(amazon_products_link)

    product_code = amazon_products_link.split("/dp/")[1].split("/")[0] 
    product_codes.append(product_code)
    amazon_driver.get(amazon_products_links[0])
    valid_product = True
    for i in range(0,5):
        try: 
            html_source = amazon_driver.page_source
            soup = BeautifulSoup(html_source, "html.parser")
                    # Try to find the element using its XPATH
            priceTag = soup.find(id="corePriceDisplay_desktop_feature_div")
            if priceTag:
                price= priceTag.find(class_="a-price aok-align-center reinventPricePriceToPayMargin priceToPay").text

                break
            else:
                time.sleep(1)
        except Exception as e:
            print(e)

   
    print(price)

    other_sellers_button_PATH = 'a-price'
    try:
        # Wait until the element is clickable
        WebDriverWait(amazon_driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, other_sellers_button_PATH))
        )
        amazon_driver.find_element(By.XPATH, other_sellers_button_PATH).click()
    except Exception as e:
        print(f"An error occurred: {e}")
        time.sleep(0.5)

    for i in range(0,5):
        try: 
            html_source = amazon_driver.page_source
            soup = BeautifulSoup(html_source, "html.parser")
            slider = soup.find(id="all-offers-display-scroller")
            if slider:
                amazon_sellerstags = slider.find_all(class_="a-size-small a-link-normal")
                amazon_pricestags = slider.find_all(class_="a-section a-spacing-none aok-align-center aok-relative")
                break
            else:
                time.sleep(1)
        except Exception as e:
            print(e)

    

    amazon_sellers = []
    for seller in amazon_sellerstags:
        try:
            # Check if seller is valid and has the required aria-label attribute
            if seller["aria-label"] == "Opens a new page":
                amazon_sellers.append(seller.text)  # Add to the list if conditions are met
        except Exception as e:
            # Handle any exceptions (e.g., element doesn't support get_attribute)
            print(e)  # You can log the exception if needed or just pass


    def check_blacklisted_sellers(sellers_list, sellers_black_list):
        # Find the blacklisted sellers in the sellers list
        blacklisted_sellers = [seller for seller in sellers_list if seller in sellers_black_list]

        if blacklisted_sellers:
            valid_product = False
            print("Not a valid product")
        else:
            print("No sellers in the list are blacklisted.")
    
    def check_whitelisted_sellers(sellers_list, seller_white_list):
        # Loop through the sellers list to find the first whitelisted seller
        for index, seller in enumerate(sellers_list):
            if seller in seller_white_list:
                print("First whitelisted seller found at index:", index)
                return index  # Return the index of the first whitelisted seller
        
        print("No sellers in the list are whitelisted.")
        return -1  # Return -1 if no whitelisted seller is found



    # Example usage
    Wseller_index = check_whitelisted_sellers(amazon_sellers, seller_white_list)
    if Wseller_index == -1:
        valid_product = False
    sellers20pounds_counter = 0 
    amazon_prices = []
    for p in amazon_pricestags:
        priceSTR = p.find(class_='aok-offscreen').text.strip().split(" ")[0].replace("£", "")
        riceNMR = float(priceSTR.replace("£", ""))
        amazon_prices.append(riceNMR)
        if riceNMR <= 20.00:
            sellers20pounds_counter += 1
    if sellers20pounds_counter >= 2:
        valid_product = False
        print("Not a valid product")
    amazon_price = amazon_prices [Wseller_index]
    percentage = 0.01

    # Calculate 1% of the price
    reduction = amazon_price * percentage

    # Calculate the new price
    calculated_price = amazon_price - reduction
    calculated_prices.append(calculated_price)
    print("Price after reducing 1% from", amazon_price, "is", calculated_price)




    # # Removing duplicates while maintaining order using a set
    # unique_sellers = []
    # seen_sellers = set()
    # for seller in sellers:
    #     if seller not in seen_sellers:
    #         unique_sellers.append(seller)
    #         seen_sellers.add(seller)

    # unique_prices = []
    # seen_prices = set()
    # for price in prices:
    #     if price not in seen_prices:
    #         unique_prices.append(price)
    #         seen_prices.add(price)

    # print(unique_sellers)  # This will maintain the original order of sellers
    # print(unique_prices)   # This will maintain the original order of prices

    html_source = amazon_driver.page_source
    soup = BeautifulSoup(html_source, "html.parser")
    product_title = soup.find(id="productTitle").text.strip()
    amazon_titles.append(product_title)
    product_image_tag = soup.find(id='imgTagWrapperId')
    product_image = product_image_tag.find('img')['src']
    webuy_driver = create_driver_without_proxy()

    webuy_url = f"https://uk.webuy.com/search?stext={product_title.replace(' ', '+').replace('&', 'and')}"
    webuy_driver.get(webuy_url)
    time.sleep(5)
    html_source = webuy_driver.page_source
    webuy_soup = BeautifulSoup(html_source, "html.parser")
    for _ in range(0, 5):
        html_source = webuy_driver.page_source
        webuy_soup = BeautifulSoup(html_source, "html.parser")


        cookies = webuy_soup.find(id="onetrust-accept-btn-handler")
        cookies_path = '/html/body/div[3]/div[2]/div/div/div[2]/div/div/button'
        if cookies:
            print("cookies found")
            wait_for_element_to_be_clickable(webuy_driver, cookies_path).click()

            break
        else:
            time.sleep(1)
    for _ in range(0, 5):


        html_source = webuy_driver.page_source
        webuy_soup = BeautifulSoup(html_source, "html.parser")
        results = webuy_soup.find(class_='search-result-grid mb-s md-mb-0')
        if results:
            break
        else:
            time.sleep(1)
    html_source = webuy_driver.page_source
    webuy_soup = BeautifulSoup(html_source, "html.parser")
    results = webuy_soup.find(class_='search-result-grid mb-s md-mb-0')

    webuy_products = results.find_all(class_='search-product-card')

    webuy_product_titles = []
    webuy_product_links = []
    webuy_product_images = []
    for product in webuy_products:
        webuy_product_titles.append(product.find(class_='card-title').text)
        webuy_product_links.append('https://uk.webuy.com' + product.find('a')['href'])
        webuy_product_images.append(product.find('img')['src'])




    def calculate_similarity(reference_string, string_list):
        similarity_scores = []
        
        for string in string_list:
            # Calculate similarity ratio
            similarity_ratio = difflib.SequenceMatcher(None, reference_string, string).ratio()
            # Convert to percentage
            similarity_percentage = similarity_ratio * 100
            similarity_scores.append((string, similarity_percentage))
        
        return similarity_scores

    title_similarity_results = calculate_similarity(product_title, webuy_product_titles)

    # Print results
    for string, score in title_similarity_results:
        print(f"Similarity with '{string}': {score:.2f}%")






    print("Libraries are installed correctly!")

    def download_images(image_urls, download_dir='images\\temporary_images'):
        """Download images from the given URLs."""
        if not os.path.exists(download_dir):
            os.makedirs(download_dir)

        downloaded_images = []
        for url in image_urls:
            try:
                response = requests.get(url)
                response.raise_for_status()  # Raise an error for bad responses
                image_name = os.path.join(download_dir, url.split("/")[-1])
                with open(image_name, 'wb') as file:
                    file.write(response.content)
                downloaded_images.append(image_name)
            except Exception as e:
                print(f"Error downloading {url}: {e}")

        return downloaded_images


    def download_reference_image(url, download_dir='images\\reference'):
        """Download the reference image to a specified directory."""
        if not os.path.exists(download_dir):
            os.makedirs(download_dir)
        
        try:
            response = requests.get(url)
            response.raise_for_status()  # Raise an error for bad responses
            image_name = os.path.join(download_dir, url.split("/")[-1])
            with open(image_name, 'wb') as file:
                file.write(response.content)
            return image_name
        except Exception as e:
            print(f"Error downloading reference image {url}: {e}")
            return None


    def calculate_ssim(img1, img2):
        """Calculate SSIM manually."""
        # Constants to avoid division by zero
        C1 = 6.5025
        C2 = 58.5225

        # Calculate means
        mu1 = np.mean(img1)
        mu2 = np.mean(img2)

        # Calculate variances
        sigma1_sq = np.var(img1)
        sigma2_sq = np.var(img2)
        sigma12 = np.cov(img1.flatten(), img2.flatten())[0][1]

        # SSIM formula
        ssim = ((2 * mu1 * mu2 + C1) * (2 * sigma12 + C2)) / \
                ((mu1 ** 2 + mu2 ** 2 + C1) * (sigma1_sq + sigma2_sq + C2))

        return ssim


    def compare_images(image1_path, image2_path):
        """Compare two images and return similarity percentage using OpenCV."""
        # Read the images
        img1 = cv2.imread(image1_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
        img2 = cv2.imread(image2_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale

        # Resize images to the same size for comparison
        img1 = cv2.resize(img1, (100, 100))  # Adjust size as necessary
        img2 = cv2.resize(img2, (100, 100))  # Adjust size as necessary

        # Calculate SSIM
        ssim_score = calculate_ssim(img1, img2)
        
        # Convert to percentage
        similarity_percentage = ssim_score * 100
        return similarity_percentage


    # Download images
    downloaded_images = download_images(webuy_product_images)

    # Download the reference image
    reference_image_path = download_reference_image(product_image)
    amazon_images.append(reference_image_path)
    # Compare each downloaded image with the reference image if it was successfully downloaded
    if reference_image_path:
        image_similarity_results = []
        for img_path in downloaded_images:
            similarity = compare_images(reference_image_path, img_path)
            image_similarity_results.append((img_path, similarity))

        # Print results
        for img_path, score in image_similarity_results:
            print(f"Similarity with '{img_path}': {score:.2f}%")
    else:
        print("Reference image download failed; comparison will not be performed.")





    def calculate_percentages(title_data, image_data):
        results = []
        
        # Ensure both lists have the same length
        if len(title_data) != len(image_data):
            raise ValueError("The length of title_data and image_data must be the same.")
        
        for (title, title_percentage), (image, image_percentage) in zip(title_data, image_data):
            new_percentage = (0.8 * image_percentage) + (0.2 * title_percentage)
            results.append(new_percentage)
        
        return results


    # Calculate percentages
    percentages = calculate_percentages(title_similarity_results, image_similarity_results)


    # Finding the highest percentage and its index
    highest_percentage = max(percentages)
    highest_index = percentages.index(highest_percentage)
    webuy_image = image_similarity_results[percentages.index(highest_percentage)][0]
    webuy_images.append(webuy_image)
    webuy_link = webuy_product_links[highest_index]
    webuy_driver.get(webuy_link)
    webuy_links.append(webuy_link)
    def clear_temp_images(download_dir='images\\temporary_images'):
        """Clear all images in the temporary images directory."""
        if os.path.exists(download_dir):
            for filename in os.listdir(download_dir):
                file_path = os.path.join(download_dir, filename)
                try:
                    if os.path.isfile(file_path):
                        if file_path != webuy_image:
                            os.remove(file_path)  # Remove the file
                            print(f"Deleted: {file_path}")
                except Exception as e:
                    print(f"Error deleting {file_path}: {e}")
        else:
            print(f"The directory {download_dir} does not exist.")

    # Example usage
    clear_temp_images()
    time.sleep(2)
    for _ in range(0, 5):
        html_source = webuy_driver.page_source
        webuy_soup = BeautifulSoup(html_source, "html.parser")

        cookies = webuy_soup.find(id="onetrust-accept-btn-handler")
        cookies_path = '/html/body/div[3]/div[2]/div/div/div[2]/div/div/button'
        if cookies:
            print("cookies found")
            wait_for_element_to_be_clickable(webuy_driver, cookies_path).click()

            break
        else:
            time.sleep(1)

    for _ in range(0, 5):
        html_source = webuy_driver.page_source
        webuy_soup = BeautifulSoup(html_source, "html.parser")
        # Extracting the price
        if webuy_soup.find(class_="sell-price md-mb-0 d-block w-100") and webuy_soup.find(class_="heading-s-semibold md-heading-l-semibold"):
            webuy_priceSTR = webuy_soup.find(class_="sell-price md-mb-0 d-block w-100").text.replace("£", "")
            webuy_title = webuy_soup.find(class_="heading-s-semibold md-heading-l-semibold").text
            break        
        else:
            time.sleep(1)
    # Converting the price to a float and adding 3
    webuy_price = float(webuy_priceSTR) + 3
    formatted_price = f"£{webuy_price:.2f}"

    buy_prices.append(float(f"{webuy_price:.2f}"))
    webuy_titles.append(webuy_title)
    # Alternatively, using an f-string

    # Printing the formatted price
    print("The formatted price is:", formatted_price)
    for _ in range(0, 5):
        check_store_button_PATH = '/html/body/div[1]/div/main/div/div/div[1]/div[1]/div[2]/div[4]/div[2]/div/span'

        try:
            # Wait until the element is clickable
            WebDriverWait(webuy_driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, check_store_button_PATH))
            )
            webuy_driver.find_element(By.XPATH, check_store_button_PATH).click()
        except Exception as e:
            print(f"An error occurred: {e}")
            time.sleep(0.5)

    for _ in range(0, 5):
        
        all_store_button_PATH = '/html/body/div[1]/div/main/div/div/div[6]/div[2]/button[2]/span'

        try:
            # Wait until the element is clickable
            WebDriverWait(webuy_driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, all_store_button_PATH))
            )
            webuy_driver.find_element(By.XPATH, all_store_button_PATH).click()
        except Exception as e:
            print(f"An error occurred: {e}")
            time.sleep(0.5)
    html_source = webuy_driver.page_source
    webuy_soup = BeautifulSoup(html_source, "html.parser")
    store_name_class = 'store-name d-flex justify-content-between'
    for _ in range(0, 5):
        if len(webuy_soup.find_all(class_=store_name_class)) < 3:
            valid_product = False
            print("Not a valid product")


    store = webuy_soup.find_all(class_=store_name_class)
    comparing_price = webuy_price *2
    cheap_sellers = 0
    for price in amazon_prices:
        if price < comparing_price:
            cheap_sellers += 1

    if cheap_sellers >= 3:
        valid_product = False
        print("Not a valid product")


    break

An error occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6999EB645+29573]
	(No symbol) [0x00007FF699960470]
	(No symbol) [0x00007FF69981B6EA]
	(No symbol) [0x00007FF69986F815]
	(No symbol) [0x00007FF69986FA6C]
	(No symbol) [0x00007FF6998BB917]
	(No symbol) [0x00007FF69989733F]
	(No symbol) [0x00007FF6998B86BC]
	(No symbol) [0x00007FF6998970A3]
	(No symbol) [0x00007FF6998612DF]
	(No symbol) [0x00007FF699862441]
	GetHandleVerifier [0x00007FF699D1C58D+3375821]
	GetHandleVerifier [0x00007FF699D67987+3684039]
	GetHandleVerifier [0x00007FF699D5CDAB+3640043]
	GetHandleVerifier [0x00007FF699AAB7C6+816390]
	(No symbol) [0x00007FF69996B77F]
	(No symbol) [0x00007FF6999675A4]
	(No symbol) [0x00007FF699967740]
	(No symbol) [0x00007FF69995659F]
	BaseThreadInitThunk [0x00007FFAAFD97374+20]
	RtlUserThreadStart [0x00007FFAB163CC91+33]

2000
20
 £30.98
An error occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6999EB645+29573]
	(No symbol) [0x00007FF699960470]
	(No symbol) [0x000

In [60]:
import openpyxl
from openpyxl.drawing.image import Image
from openpyxl.styles import Font, PatternFill

data = {
    'Amazon_title': amazon_titles,
    'Amazon_link': amazon_links,
    'Webuy_title': webuy_titles,
    'Webuy_link': webuy_links,
    'Amazon_image': amazon_images,
    'Webuy_image': webuy_images,
    'Calculated_price': calculated_prices,
    'Product_code': product_codes,
    'Buy_price': buy_prices,
}

# Create a new workbook and select the active worksheet
wb = openpyxl.Workbook()
ws = wb.active
ws.title = "Products"

# Write headers
headers = ['Amazon Title', 'Amazon Link', 'Webuy Title', 'Webuy Link', 'Amazon Image', 'Webuy Image','Calculated Price', 'Product Code',  'Buy Price']
ws.append(headers)

# Define styles for headers
header_fill = PatternFill(start_color="808080", end_color="808080", fill_type="solid")  # Gray fill
header_font = Font(bold=True)  # Bold font

# Apply styles to headers
for col_num, header in enumerate(headers, start=1):
    cell = ws.cell(row=1, column=col_num)
    cell.fill = header_fill
    cell.font = header_font

# Set column widths
column_widths = [30, 20, 30, 20, 10, 10, 15, 15, 15]  # Customize widths as needed
for col_num, width in enumerate(column_widths, start=1):
    ws.column_dimensions[openpyxl.utils.get_column_letter(col_num)].width = width

# Add data and images
for i in range(len(data['Amazon_title'])):
    # Append Amazon Title
    ws.cell(row=i + 2, column=1, value=data['Amazon_title'][i])  # Amazon Title

    # Create clickable hyperlink for Amazon link
    amazon_cell = ws.cell(row=i + 2, column=2)
    amazon_cell.value = "Amazon Link"  # Set display text
    amazon_cell.hyperlink = data['Amazon_link'][i]  # Directly assign the link

    # Append Webuy Title
    ws.cell(row=i + 2, column=3, value=data['Webuy_title'][i])  # Webuy Title

    # Create clickable hyperlink for Webuy link
    webuy_cell = ws.cell(row=i + 2, column=4)
    webuy_cell.value = "Webuy Link"  # Set display text
    webuy_cell.hyperlink = data['Webuy_link'][i]  # Directly assign the link

    # Load and insert the Amazon image
    amazon_img_path = data['Amazon_image'][i]
    amazon_img = Image(amazon_img_path)
    amazon_img.width = 50  # Set width
    amazon_img.height = 50  # Set height
    amazon_img.anchor = f'E{i + 2}'  # Column E for Amazon Images
    ws.add_image(amazon_img)

    # Load and insert the Webuy image
    webuy_img_path = data['Webuy_image'][i]
    webuy_img = Image(webuy_img_path)
    webuy_img.width = 50  # Set width
    webuy_img.height = 50  # Set height
    webuy_img.anchor = f'F{i + 2}'  # Column F for Webuy Images
    ws.add_image(webuy_img)

    # Set row height to fit the images
    ws.row_dimensions[i + 2].height = amazon_img.height  # Set height based on Amazon image

    # Product Code, Calculated Price, and Buy Price
    ws.cell(row=i + 2, column=7, value=f"£{data['Calculated_price'][i]:.2f}")  # Calculated Price
    ws.cell(row=i + 2, column=8, value=data['Product_code'][i])  # Product Code
    ws.cell(row=i + 2, column=9, value=f"£{data['Buy_price'][i]:.2f}")  # Buy Price

    # Fill data cells with color
    for col_num in range(1, len(headers) + 1):
        cell = ws.cell(row=i + 2, column=col_num)
        cell.fill = PatternFill(start_color="D3D3D3", end_color="D3D3D3", fill_type="solid")  # Light gray fill

# Save the workbook
wb.save('final_sheet.xlsx')


In [181]:
webuy_links

[]

In [ ]:
import pyautogui
import time

# Step 1: Move the mouse to the pixel coordinates and click
x = 1415  # Replace with the X coordinate of the pixel
y = 406  # Replace with the Y coordinate of the pixel

# Move the mouse to the (x, y) position and click
pyautogui.moveTo(x, y)
pyautogui.click()

# Optional: You can add a delay before sending keys
time.sleep(1)

# Step 2: Send keys (simulates typing a string)
pyautogui.write("29", interval=0.1)  # Types 'Hello World' with a slight delay between each character

# Step 3: Press individual keys (simulates key presses)
pyautogui.press('enter')  # Simulates pressing the 'Enter' key



In [36]:
for i in range(0,5):
    try: 
        html_source = amazon_driver.page_source
        soup = BeautifulSoup(html_source, "html.parser")
        slider = soup.find(id="all-offers-display-scroller")
        if slider:
            break
        else:
            time.sleep(1)
    except Exception as e:
        print(e)

amazon_sellerstags = slider.find_all(class_="a-size-small a-link-normal")
amazon_pricestags = slider.find_all(class_="a-section a-spacing-none aok-align-center aok-relative")

In [41]:
amazon_sellers

[' Selling-Prime-UK',
 ' Selling-Prime-UK',
 ' Brit_Books',
 ' musicMagpie',
 ' moviemars-uk',
 ' Davina Jayne Ltd',
 ' Chalkys UK',
 ' Marastore',
 ' RAREWAVES',
 ' Red Pebble Products',
 ' reflexcd2 SHIPPING FROM THE UK']

In [43]:
webuy_driver = create_driver_without_proxy()


'Incredibles, The - 2 Disc SE (U) 2004'